## Convolutional Neural Network

In [1]:
import numpy as np

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

from sklearn.metrics import classification_report

Using TensorFlow backend.


## Retrieve all of the features and labels

In [2]:
npzfile = np.load('../data/melspects.npz', allow_pickle=True)

In [3]:
X_train, y_train = npzfile['X_train'], npzfile['y_train']
X_test, y_test = npzfile['X_test'], npzfile['y_test']

In [4]:
X_test.shape

(250, 96, 640)

## Building the Neural Net

In [5]:
def CNN(X_shape, nb_classes):
    
    # Input
    inputs = Input(shape=X_shape)
    
    conv1 = Conv2D(filters=64, kernel_size=(4,4), strides=1, padding='valid', 
                   activation='relu')(inputs)
    
    b1 = BatchNormalization()(conv1)
    
    pool1 = MaxPooling2D((2, 4), strides=(2, 4))(b1)
    
    conv2 = Conv2D(filters=64, kernel_size=(3,5), strides=1, padding='valid', 
                   activation='relu')(pool1)
    
    pool2 = MaxPooling2D((2, 2), strides=(2, 2))(conv2)
    
    dr1 = Dropout(0.2)(pool2)
    
    conv3 = Conv2D(filters=64, kernel_size=(2,2), strides=1, padding='valid', 
                   activation='relu')(dr1)
    
    b2 = BatchNormalization()(conv3)
    
    pool3 = MaxPooling2D((2, 2), strides=(2, 4))(b2)
    
    flatten = Flatten()(pool3)
    
    dense1 = Dense(64, activation='relu')(flatten)
    
    dense2 = Dense(32, activation='relu')(dense1)
    
    # Softmax Output
    output = Dense(nb_classes, activation='softmax')(dense2)
    
    model_output = output
    model = Model(inputs=inputs, outputs=[output])
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    print(model.summary())
    return model

In [ ]:
nb_classes = len(np.unique(y_train))
nb_epochs = 25
batch_size = 128

# Convert to one hot encoding
y_train_one_hot = np.zeros((y_train.size, nb_classes))
y_train_one_hot[range(y_train.size),y_train] = 1

# Need to add a channel dim for the convolution layers
X_train_expanded = np.expand_dims(X_train, axis=-1)
X_test_expanded = np.expand_dims(X_test, axis=-1)

X_shape = X_train_expanded.shape[1:]

model = CNN(X_shape, nb_classes)

# Fit data to model
history = model.fit(X_train_expanded, y_train_one_hot, batch_size=batch_size, 
                    epochs=nb_epochs, validation_split=0.1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 640, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 93, 637, 64)       1088      
_________________________________________________________________
batch_normalization_1 (Batch (None, 93, 637, 64)       256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 46, 159, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 155, 64)       61504     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 77, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 77, 64)        0   

In [34]:
preds = model.predict(X_test_expanded)
y_pred = np.argmax(preds, axis=1)

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.43      0.35      0.38        26
           1       1.00      0.31      0.47        29
           2       0.33      0.43      0.38        23
           3       0.64      0.36      0.46        25
           4       0.44      0.85      0.58        27
           5       0.52      0.56      0.54        25
           6       0.86      0.86      0.86        22
           7       0.86      0.43      0.57        28
           8       0.29      0.48      0.36        21
           9       0.37      0.42      0.39        24

    accuracy                           0.50       250
   macro avg       0.58      0.50      0.50       250
weighted avg       0.59      0.50      0.50       250

